In [1]:
def point_torsion(P,Q,baseorder,exporder,*arg): 
    
    '''
    Returns a point S of order (baseorder)^exporder from torsion group generated by P and Q. It also returns tb 
    such that S=P+tb·Q. If args are added it also returns mb and nb such that S = mb·P+nb·Q
    
    ''' 
    
    r=1                                           
    order=baseorder^exporder                     
    while r:
        mb=P.curve.F(randint(0,order-1))
        try:
            if int(pow(mb,-1))%1==0:
                r=0
        except:
            pass
    mbinv= P.curve.F(pow(mb,-1))
    r=1
    while r:
        nb=randint(0,order-1)
        tb=P.curve.F(mbinv*nb)
        S1=Q*tb
        S=P+S1
        if(S.order_torsion(baseorder,exporder)==order):
            r=0
    if(arg is not None):
        return S,tb,mb,nb
    else:
        return S,tb

In [2]:
def check_order_F(P,n):
    '''
    INPUT: an element P from a finite field and an integer n st order(P)=n^a for some integer a.
    OUTPUT: order of P in F.
    '''
    
    i = 0               
    while(i >= 0):
        P = P^n
        if P == 1:
            return i+1
        i = i + 1

In [3]:
def curve_2_iso(P,B):
    '''              
    INPUT: P point on E of order 2, B coefficient from curve E in Montgomery form
    OUTPUT: A and B from image curve of the isogeny with kernel generated by P
    '''
    
    a=2*(P.curve.F(1)-2*pow(P.x,2))
    b=P.x*B
    return (P.curve.F(a),P.curve.F(b))

In [4]:
def eval_2_iso(P,x1):  
    '''
    Evaluation of isogeny of degree 2
    INPUT: x1 = x-coordinate of the generator of the kernel, and P point where the isogeny has to be evaluated
    OUTPUT: coordinates of the evaluation of the isogeny at P
    '''
    x2 = (pow(P.x,2)*x1-P.x)/(P.x-x1)
    y2 = P.y*(pow(P.x,2)*x1-2*P.x*pow(x1,2)+x1)/pow(P.x-x1,2)
    return (x2,y2)

In [5]:
def curve_3_iso(P,A,B): 
    '''
    INPUT: P point on E of order 3, A and B coefficients of E in Montgomery form
    OUTPUT: A and B from image curve of the isogeny with kernel generated by P
    '''
    
    a = (A*P.x-6*pow(P.x,2)+6)*P.x 
    b = B*pow(P.x,2)
    return (P.curve.F(a),P.curve.F(b))

In [6]:
def eval_3_iso(P,x):  
    '''
    Evaluation of isogeny of degree 3   
    INPUT: x= x-coordinate of the generator of the kernel and P point where the isogeny has to be evaluated
    OUTPUT: coordinates of the evaluation of the isogeny at P
    '''
    
    x2 = P.x*pow(P.x*x-1,2)/pow(P.x-x,2)
    y2 = P.y*((P.x*x - 1)*(pow(P.x,2)*x - 3*P.x*pow(x,2) + P.x + x)/(pow(P.x-x,3)))
    return (x2,y2)

In [7]:
def curve_4_iso(P,B):
    '''
    INPUT: P point on E of order 4,  B coefficient of E in Montgomery form
    OUTPUT: A and B from image curve of the isogeny with kernel generated by P
    '''
    
    a = 4*pow(P.x,4)-P.curve.F(2)
    b = -P.x*(pow(P.x,2)+P.curve.F(1))*(B*pow(P.curve.F(2),-1))
    return (P.curve.F(a),P.curve.F(b))

In [8]:
def eval_4_iso(P,x):
    '''        
    Evaluation of isogeny of degree 4   
    INPUT: x= x-coordinate of the generator of the kernel and P point where the isogeny has to be evaluated
    OUTPUT: coordinates of the evaluation of the isogeny at P
    '''
    x2 = -(P.x*pow(x,2) + P.x - 2*x)*P.x*pow(P.x*x - 1,2)/(pow(P.x - x,2)*(2*P.x*x - pow(x,2)-1))
    y2 = P.y*(-2*pow(x,2)*(P.x*x-1)*(pow(P.x,4)*(pow(x,2)+1)-4*pow(P.x,3)*(pow(x,3)+x)+2*pow(P.x,2)*(pow(x,4)+5*pow(x,2))-4*P.x*(pow(x,3)+x)+pow(x,2)+1))/(pow(P.x-x,3)*pow(2*P.x*x-pow(x,2)-1,2))
    return (x2,y2)

In [9]:
def iso_2_e(S,e,A,B,*args): 
    ''' 
    INPUT: S point of order (2^e) with e integer from elliptic curve with coefficients A and B. OPTIONAL: more
            points on the same curve as S
    OUTPUT: image curve of the isogeny with kernel generated by S (order 2^e). OPTIONAL: evaluation of the isogeny
            at the optional points.
    '''
    
    e2=e                               
    A2=A                               
    B2=B
    Punto=[args[i] for i in range(len(args))]
    if e2%2==1:
        Pt=S*pow(2,e2-1)
        A2=curve_2_iso(Pt,B2)[0]
        B2=curve_2_iso(Pt,B2)[1]
        E1=MontgomeryEC(A2,B2,F)
        S=eval_2_iso(S,Pt.x)
        S=Point(E1,S[0],S[1])
        for i in range(len(args)):
            Punto[i]=eval_2_iso(Punto[i],Pt.x)
            Punto[i]=Point(E1,Punto[i][0],Punto[i][1])
        e2=e2-1
    for i in range(e2-2,-2,-2):
        Pt=S*pow(2,i)
        A2=curve_4_iso(Pt,B2)[0]
        B2=curve_4_iso(Pt,B2)[1]
        E1=MontgomeryEC(A2,B2,F)       
        if i!=0:
            S=eval_4_iso(S,Pt.x)
            S=Point(E1,S[0],S[1])
        for j in range(len(args)):
            Punto[j]=eval_4_iso(Punto[j],Pt.x)
            Punto[j]=Point(E1,Punto[j][0],Punto[j][1])
    return (A2,B2),Punto

In [14]:
def iso_3_e(S,e,A,B,*args): 
    '''
    INPUT: S point of order (3^e) with e integer from elliptic curve with coefficients A and B. OPTIONAL: more
            points on the same curve as S
    OUTPUT: image curve of the isogeny with kernel generated by S (order 3^e). OPTIONAL: evaluation of the isogeny
            at the optional points.
    '''
    A2=A 
    B2=B
    Punto=[args[i] for i in range(len(args))]
    for i in range(e-1,-1,-1):
        Pt=S*pow(3,i)
        A2=curve_3_iso(Pt,A2,B2)[0]
        B2=curve_3_iso(Pt,A2,B2)[1]
        E1=MontgomeryEC(A2,B2,F)
        if i!=0:
            S=eval_3_iso(S,Pt.x)
            S=Point(E1,S[0],S[1])
        for j in range(len(args)):
            Punto[j]=eval_3_iso(Punto[j],Pt.x)
            Punto[j]=Point(E1,Punto[j][0],Punto[j][1])
    return (A2,B2),Punto

In [11]:
class MontgomeryEC:
    '''
    Elliptic curve in Montgomery form over a field F
    '''
    
    def __init__(self,a,b,F):
        self.a = a
        self.b = b
        self.F = F
    
    def j_invariant(self):
        '''
        Returns j-invariant of the curve
        '''
        
        a=self.a
        j = self.F(256)*pow(pow(a,2)-3,3)/(pow(a,2)-self.F(4))
        return j
    def random_point(self):
        '''
        Returns a random point from the curve
        '''
        while True:
            punto=self.F.random_element()
            eq= self.F((pow(punto,3) + self.a * pow(punto,2) + punto)/self.b)
            try:
                y=sqrt(eq)
                break
            except:
                pass
        b=randint(0,1)
        return Point(self,punto,pow(-1,b)*y)
    def testPoint(self, x, y):
        '''
        Returns true a given point is on the curve, false otherwise
        '''
        return self.b*pow(y,2) == pow(x,3) + self.a * pow(x,2) + x
    def __str__(self):
        return '%Gy^2 = x^3 + %Gx^2 + x' % (self.b, self.a)




In [12]:
class Point(object):
    '''
    Point on an elliptic curve from class MontgomeryEC
    '''
    
    def __init__(self, curve, x, y):
        self.curve = curve # the curve containing this point
        self.x = x
        self.y = y
 
        if self.x != infinity and not curve.testPoint(x,y):  
            raise Exception("The point %s is not on the given curve %s" % (self, curve))
            
    def __str__(self):
        return '('+str(self.x)+','+str(self.y)+')'
    
    def dbl(self):
        if self.x==infinity:
            return Point(self.curve,infinity,infinity)
        if (self.x==0 and self.y==0):
            return Point(self.curve,infinity,infinity)
        if (self.y==0):
            return Point(self.curve,infinity,infinity)
        
        x2 = pow(pow(self.x,2)-1,2)/(self.curve.F(4)*self.x*(pow(self.x,2)+self.curve.a*self.x + self.curve.F(1)))
        y2 = self.y*((pow(self.x,2)-1)*(pow(self.x,4)+self.curve.F(2)*self.curve.a*pow(self.x,3)+self.curve.F(6)*pow(self.x,2)+self.curve.F(2)*self.curve.a*self.x+1))/(self.curve.F(8)*pow(self.x,2)*pow(pow(self.x,2)+self.curve.a*self.x+1,2))
        return Point(self.curve,self.curve.F(x2),self.curve.F(y2))
    
    def __add__(self,other):
        if self.x==infinity:
            return other
        if other.x==infinity:
            return self
        if self.curve.F(self.x)==self.curve.F(other.x) and self.curve.F(self.y)==self.curve.F(other.y):
            return self.dbl()
        if self.x==other.x and self.curve.F(self.y)==self.curve.F(-other.y):
            return Point(self.curve,infinity,infinity)
        
        lbda = (self.y - other.y)/(self.x - other.x)
        x = self.curve.b*pow(lbda,2) - (self.x + other.x) - self.curve.a
        y = lbda*(self.x - x) - self.y
        return Point(self.curve,x,y)
    
    def __mul__(self,n):
        if int(n)%1 != 0:
            raise Exception("Can't scale a point by something which isn't an int!")
        if n==2:
            return self.dbl()
        elif n==3:
            P1=self.dbl()
            P2=P1+self
            return Point(self.curve,P2.x,P2.y)
        else:
            P0=Point(self.curve,0,0)
            tb=bin(int(n))[2:]
            for i in range(len(tb)):
                    ti=tb[i]
                    P0 = P0.dbl()
                    if int(ti)==1:
                        P0=P0+self
            return P0
    
    def __rmul__(self,n):
        return(self.__mul__(n))
    
    def __eq__(self,other):
            
        if (self.x == other.x and self.y==other.y):
            return True
        else:
            return False
    def __ne__(self, other):
         return not self == other
    def __neg__(self):
        x=self.x
        y=-self.y
        return(Point(self.curve,x,y))
    def __sub__(self,other):
        return(self+ -other)
        
    def order_torsion(self,base,exp):
        '''
        Returns the order of the point, knowing that it is a base^exp torsion point
        '''
        Punto=self
        for i in range(exp):
            Punto=Punto*base
            if Punto.x==infinity:
                return pow(base,i+1)
        raise ValueError("The point is not on the given torsion point")
    
    def slope(self,other):
        '''
        Returns the slope of the line through self and other. It will be used for the Miller algorithm. 
        '''
        if self.x == infinity or other.x == infinity:
            return infinity
        elif self == other:
            if self.y ==0:
                return infinity
            else:
                dy= 3*pow(self.x,2)+2*self.curve.a*self.x+1
                dx=2*self.curve.b*self.y
                return self.curve.F(dy/dx)
        elif self.x==other.x:
            return infinity
        
        else:
            dy= other.y-self.y
            dx=other.x-self.x
            return self.curve.F(dy/dx)
        
    def l(self,other):
        '''
        Equation of the line through self and other. It will be used for the Miller algorithm. 
        '''
        lbd=self.slope(other)
        if self.x == infinity or other.x == infinity:
            if self.x == infinity and other.x==infinity:
                def f(X,Y):
                    return 1
                return f
            elif self.x==infinity:
                def f(X,Y):
                    return X - other.x
                return f
        elif lbd == infinity:
            if self.x == infinity or self.x == -infinity:
                self.x = 0
            def f(X,Y):
                return X - self.x
            return f
        else:
            def f(X,Y):
                nom = Y - self.y - lbd*(X-self.x)
                den = 1
                #den = X + self.x + other.x - pow(lbd,2)
                return self.curve.F(nom/den)
            return f

    def miller(self,other,n):
        '''
        Miller algorithm to find f such that div(f)=n[self]-n[O] (directly evaluated at other)
        '''
        T=self
        f=self.curve.F(1)
        nbin = n.bits()
        for i in range(len(nbin)-2,-1,-1):
            S=2*T
            num = T.l(T)(other.x,other.y)
            den = S.l(-S)(other.x,other.y)
            f=f*f*self.curve.F(num/den)
            T=2*T
            if nbin[i]==1:
                S = T + self
                num = T.l(self)(other.x,other.y)
                den = S.l(-S)(other.x,other.y)
                f=f*self.curve.F(num/den)
                T = T + self
        return f

    def Weil_pairing(self,other,n):
        '''
        Returns the Weil pairing of self and other
        '''
        if self.x==0 and self.y==0:
            raise ValueError("Els punts han de ser diferents de zero")
        elif other.x==0 and other.y==0:
            raise ValueError("Els punts han de ser diferents de zero")
        else:
            S=self.curve.random_point()
            if S != self and S.x!= infinity and S != -other and S != (self - other):
                    f1=self.miller(other+S,n)
                    f2=self.miller(S,n)
                    f3=other.miller(self-S,n)
                    f4=other.miller(-S,n)
                    weil= (f1/f2)/(f3/f4)
                    return weil